# 如何将语义搜索添加到代理的内存中

本指南介绍如何在代理的内存存储中启用语义搜索。这使您的代理可以通过语义相似性在长期记忆存储中搜索项目。

## 依赖关系和环境设置

首先，安装本指南所需的依赖项。
```bash
npm install \
  @langchain/langgraph \
  @langchain/openai \
  @langchain/core \
  uuid \
  zod
```
接下来，我们需要为 OpenAI 设置 API 密钥（我们将使用的 LLM）
```bash
export OPENAI_API_KEY=your-api-key
```
或者，我们可以为 [LangSmith 追踪](https://smith.langchain.com/) 设置 API 密钥，这将为我们提供一流的可观察性。
```bash
export LANGCHAIN_TRACING_V2="true"
export LANGCHAIN_CALLBACKS_BACKGROUND="true"
export LANGCHAIN_API_KEY=your-api-key
```

## 使用语义搜索初始化内存存储

在这里，我们使用[索引配置](https://langchain-ai.github.io/langgraphjs/reference/interfaces/checkpoint.IndexConfig.html)创建内存存储。

默认情况下，商店配置为没有语义/向量搜索。您可以在创建商店时通过向商店的构造函数提供 [IndexConfig](https://langchain-ai.github.io/langgraphjs/reference/interfaces/checkpoint.IndexConfig.html) 来选择对项目建立索引。

如果您的存储类没有实现此接口，或者您没有传入索引配置，则语义搜索将被禁用，并且传递给 `put` 的所有 `index` 参数将无效。

现在，让我们创建该商店！

In [1]:
import { OpenAIEmbeddings } from "@langchain/openai";
import { InMemoryStore } from "@langchain/langgraph";

const embeddings = new OpenAIEmbeddings({
  model: "text-embedding-3-small",
});

const store = new InMemoryStore({
  index: {
    embeddings,
    dims: 1536,
  }
});

## 内存的解剖

在我们进入语义搜索之前，让我们先看看记忆是如何构造的，以及如何存储它们：

In [2]:
let namespace = ["user_123", "memories"]
let memoryKey = "favorite_food"
let memoryValue = {"text": "I love pizza"}

await store.put(namespace, memoryKey, memoryValue)

如您所见，内存由名称空间、键和值组成。

**命名空间**是多维值（字符串数组），允许您根据应用程序的需要对内存进行分段。在本例中，我们使用用户 ID (`"user_123"`) 作为命名空间数组的第一个维度，按用户对内存进行分段。

**键**是标识命名空间内的内存的任意字符串。如果多次写入同一命名空间中的同一键，则会覆盖存储在该键下的内存。

**值**是代表实际存储的内存的对象。这些可以是任何对象，只要它是可序列化的。您可以根据应用程序的需要构建这些对象。

## 简单的内存检索

让我们向我们的存储添加更多内存，然后通过它的键获取其中一个内存以检查它是否正确存储。

In [3]:
await store.put(
  ["user_123", "memories"],
  "italian_food",
  {"text": "I prefer Italian food"}
)
await store.put(
  ["user_123", "memories"],
  "spicy_food",
  {"text": "I don't like spicy food"}
)
await store.put(
  ["user_123", "memories"],
  "occupation",
  {"text": "I am an airline pilot"}
)

// 这个职业太崇高了，我们重写吧
// 还有更多……脚踏实地
await store.put(
  ["user_123", "memories"],
  "occupation",
  {"text": "I am a tunnel engineer"}
)

// 现在让我们检查一下我们的占用内存是否被覆盖了
const occupation = await store.get(["user_123", "memories"], "occupation")
console.log(occupation.value.text)

I am a tunnel engineer


## 用自然语言搜索记忆

现在我们已经了解了如何通过命名空间和键来存储和检索记忆，让我们看看如何使用语义搜索来检索记忆。

想象一下，我们有一大堆想要搜索的记忆，但我们不知道与我们想要检索的记忆相对应的密钥。语义搜索允许我们通过使用文本嵌入执行自然语言查询来搜索没有键的内存存储。我们在以下示例中演示了这一点：

In [4]:
const memories = await store.search(["user_123", "memories"], {
  query: "What is my occupation?",
  limit: 3,
});

for (const memory of memories) {
  console.log(`Memory: ${memory.value.text} (similarity: ${memory.score})`);
}

Memory: I am a tunnel engineer (similarity: 0.3070681445327329)
Memory: I prefer Italian food (similarity: 0.1435366180543232)
Memory: I love pizza (similarity: 0.10650935500808985)


## 简单示例：ReAct 代理中的长期语义记忆

让我们看一个为智能体提供长期记忆的简单示例。

长期记忆可以分为两个阶段：存储和回忆。

在下面的示例中，我们通过为代理提供一个可用于创建新内存的工具来处理存储。

为了处理召回，我们将添加一个提示步骤，使用用户聊天消息中的文本查询内存存储。然后，我们将该查询的结果注入到系统消息中。

### 简单的内存存储工具

让我们首先创建一个让法学硕士存储新记忆的工具：

In [5]:
import { tool } from "@langchain/core/tools";
import { LangGraphRunnableConfig } from "@langchain/langgraph";

import { z } from "zod";
import { v4 as uuidv4 } from "uuid";

const upsertMemoryTool = tool(async (
  { content },
  config: LangGraphRunnableConfig
): Promise<string> => {
  const store = config.store as InMemoryStore;
  if (!store) {
    throw new Error("No store provided to tool.");
  }
  await store.put(
    ["user_123", "memories"],
    uuidv4(), // give each memory its own unique ID
    { text: content }
  );
  return "Stored memory.";
}, {
  name: "upsert_memory",
  schema: z.object({
    content: z.string().describe("The content of the memory to store."),
  }),
  description: "Upsert long-term memories.",
});

在上面的工具中，我们使用UUID作为密钥，这样内存存储就可以无限地积累内存，而不用担心密钥冲突。我们这样做而不是将内存累积到单个对象或数组中，因为内存存储按键索引项目。在存储中为每个存储器赋予其自己的密钥允许为每个存储器分配其自己的唯一嵌入向量，该嵌入向量可以与搜索查询相匹配。

### 简单的语义回忆机制

现在我们有了一个存储记忆的工具，让我们创建一个提示函数，我们可以将其与 `createReactAgent` 一起使用来处理调用机制。

请注意，如果我们在这里不使用 `createReactAgent`，您可以使用与图形中的第一个节点相同的函数，并且它也可以正常工作。

In [6]:
import { MessagesAnnotation } from "@langchain/langgraph";

const addMemories = async (
  state: typeof MessagesAnnotation.State,
  config: LangGraphRunnableConfig
) => {
  const store = config.store as InMemoryStore;

  if (!store) {
    throw new Error("No store provided to state modifier.");
  }
  
  // 根据用户的最后一条消息进行搜索
  const items = await store.search(
    ["user_123", "memories"], 
    { 
      // 假设这不是一条复杂的消息
      query: state.messages[state.messages.length - 1].content as string,
      limit: 4 
    }
  );

  
  const memories = items.length 
    ? `## Memories of user\n${
      items.map(item => `${item.value.text} (similarity: ${item.score})`).join("\n")
    }`
    : "";

  // 将检索到的记忆添加到系统消息中
  return [
    { role: "system", content: `You are a helpful assistant.\n${memories}` },
    ...state.messages
  ];
};

### 把它们放在一起

最后，让我们使用 `createReactAgent` 将它们组合到一个代理中。请注意，我们在这里没有添加检查点。下面的示例不会重用消息历史记录。输入消息中未包含的所有详细信息都将来自上面定义的召回机制。

In [7]:
import { ChatOpenAI } from "@langchain/openai";
import { createReactAgent } from "@langchain/langgraph/prebuilt";

const agent = createReactAgent({
  llm: new ChatOpenAI({ model: "gpt-4o-mini" }),
  tools: [upsertMemoryTool],
  prompt: addMemories,
  store: store
});

### 使用我们的样品代理

现在我们已经把所有东西都放在一起了，让我们测试一下！

首先，让我们定义一个辅助函数，我们可以用它来打印对话中的消息。

In [8]:
import {
  BaseMessage,
  isSystemMessage,
  isAIMessage,
  isHumanMessage,
  isToolMessage,
  AIMessage,
  HumanMessage,
  ToolMessage,
  SystemMessage,
} from "@langchain/core/messages";

function printMessages(messages: BaseMessage[]) {
  for (const message of messages) {
    if (isSystemMessage(message)) {
      const systemMessage = message as SystemMessage;
      console.log(`System: ${systemMessage.content}`);
    } else if (isHumanMessage(message)) {
      const humanMessage = message as HumanMessage;
      console.log(`User: ${humanMessage.content}`);
    } else if (isAIMessage(message)) {
      const aiMessage = message as AIMessage;
      if (aiMessage.content) {
        console.log(`Assistant: ${aiMessage.content}`);
      }
      if (aiMessage.tool_calls) {
        for (const toolCall of aiMessage.tool_calls) {
          console.log(`\t${toolCall.name}(${JSON.stringify(toolCall.args)})`);
        }
      }
    } else if (isToolMessage(message)) {
      const toolMessage = message as ToolMessage;
      console.log(
        `\t\t${toolMessage.name} -> ${JSON.stringify(toolMessage.content)}`
      );
    }
  }
}

现在，如果我们运行代理并打印消息，我们可以看到代理记住了我们在本演示开始时添加到商店的食物偏好！

In [9]:

let result = await agent.invoke({
  messages: [
    {
      role: "user",
      content: "I'm hungry. What should I eat?",
    },
  ],
});

printMessages(result.messages);

User: I'm hungry. What should I eat?
Assistant: Since you prefer Italian food and love pizza, how about ordering a pizza? You could choose a classic Margherita or customize it with your favorite toppings, making sure to keep it non-spicy. Enjoy your meal!


#### 储存新的记忆

现在我们知道召回机制是有效的，让我们看看是否可以让我们的示例代理存储新的内存：

In [10]:
result = await agent.invoke({
  messages: [
    {
      role: "user",
      content: "Please remember that every Thursday is trash day.",
    },
  ],
});

printMessages(result.messages);

User: Please remember that every Thursday is trash day.
	upsert_memory({"content":"Every Thursday is trash day."})
		upsert_memory -> "Stored memory."
Assistant: I've remembered that every Thursday is trash day!


现在它已经存储了，让我们看看它是否还记得。

请记住 - 这里没有检查点。每次我们调用代理时，这都是一次全新的对话。

In [11]:
result = await agent.invoke({
  messages: [
    {
      role: "user",
      content: "When am I supposed to take out the garbage?",
    },
  ],
});

printMessages(result.messages);

User: When am I supposed to take out the garbage?
Assistant: You take out the garbage every Thursday, as it's trash day for you.


## 高级用法

上面的示例非常简单，但希望它可以帮助您想象如何将存储和调用机制交织到代理中。在下面的部分中，我们将讨论更多主题，当您进入更高级的用例时，这些主题可能会对您有所帮助。

### 多向量索引

您可以分别存储和搜索记忆的不同方面，以提高召回率或从语义索引过程中省略某些字段。

In [12]:
import { InMemoryStore } from "@langchain/langgraph";

// 配置存储以嵌入记忆内容和情感背景
const multiVectorStore = new InMemoryStore({
  index: {
    embeddings: embeddings,
    dims: 1536,
    fields: ["memory", "emotional_context"],
  },
});

// 用不同的内容/情感对存储记忆
await multiVectorStore.put(["user_123", "memories"], "mem1", {
  memory: "Had pizza with friends at Mario's",
  emotional_context: "felt happy and connected",
  this_isnt_indexed: "I prefer ravioli though",
});
await multiVectorStore.put(["user_123", "memories"], "mem2", {
  memory: "Ate alone at home",
  emotional_context: "felt a bit lonely",
  this_isnt_indexed: "I like pie",
});

// 专注于情绪状态的搜索 - 匹配 mem2
const results = await multiVectorStore.search(["user_123", "memories"], {
  query: "times they felt isolated",
  limit: 1,
});

console.log("Expect mem 2");

for (const r of results) {
  console.log(`Item: ${r.key}; Score(${r.score})`);
  console.log(`Memory: ${r.value.memory}`);
  console.log(`Emotion: ${r.value.emotional_context}`);
}

Expect mem 2
Item: mem2; Score(0.58961641225287)
Memory: Ate alone at home
Emotion: felt a bit lonely


### 在存储时覆盖字段
无论存储的默认配置如何，您都可以在使用 `put(..., { index: [...fields] })` 存储特定内存时覆盖要嵌入的字段。

In [13]:
import { InMemoryStore } from "@langchain/langgraph";

const overrideStore = new InMemoryStore({
  index: {
    embeddings: embeddings,
    dims: 1536,
    // 默认嵌入内存字段
    fields: ["memory"],
  }
});

// 使用默认索引存储一个内存
await overrideStore.put(["user_123", "memories"], "mem1", {
  memory: "I love spicy food",
  context: "At a Thai restaurant",
});

// 存储另一个内存，覆盖要嵌入的字段
await overrideStore.put(["user_123", "memories"], "mem2", {
  memory: "I love spicy food",
  context: "At a Thai restaurant",
  // 覆盖：仅嵌入上下文
  index: ["context"]
});

// 搜索食物 - 匹配 mem1（使用默认字段）
console.log("Expect mem1");
const results2 = await overrideStore.search(["user_123", "memories"], {
  query: "what food do they like",
  limit: 1,
});

for (const r of results2) {
  console.log(`Item: ${r.key}; Score(${r.score})`);
  console.log(`Memory: ${r.value.memory}`);
}

// 搜索餐厅氛围 - 匹配 mem2（使用覆盖字段）
console.log("Expect mem2");
const results3 = await overrideStore.search(["user_123", "memories"], {
  query: "restaurant environment",
  limit: 1,
});

for (const r of results3) {
  console.log(`Item: ${r.key}; Score(${r.score})`);
  console.log(`Memory: ${r.value.memory}`);
}

Expect mem1
Item: mem1; Score(0.3375009515587189)
Memory: I love spicy food
Expect mem2
Item: mem2; Score(0.1920732213417712)
Memory: I love spicy food
